Libraries

In [42]:
import os
import glob
from deepface import DeepFace
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import wandb
from wandb.keras import WandbCallback

Actor Dataset

In [ ]:
Data, I = [], []

for i, dir in enumerate(os.listdir("./Dataset_Actor/")):
    dir = glob.glob(os.path.join(f"./Dataset_Actor/{dir}/*"))
    
    for j in dir:
        try:
            embedding = DeepFace.represent(img_path = j, model_name = 'ArcFace')    
            Data.append(embedding)
            I.append(i)
        except:
            print(f"./Dataset_Actor/{dir}/{j}")
            

Create features file

In [8]:
df = pd.DataFrame(Data)
df['target'] = I

df

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
0,-0.250973,0.561123,-0.053094,-0.410861,-0.018607,0.310215,-0.151722,-0.202542,-0.009447,-0.112865,...,0.094442,0.438000,-0.781172,0.274844,-0.264912,-0.059449,-0.007042,-0.383336,-1.143475,0
1,-0.245689,0.708880,-0.176685,-0.457115,-0.031878,0.404533,-0.227349,-0.208888,-0.017394,-0.034352,...,0.117269,0.471791,-0.844017,0.300227,-0.297739,-0.103941,0.042201,-0.470301,-1.371787,0
2,-0.230934,0.612900,-0.192643,-0.514945,0.050534,0.365307,-0.227889,-0.090390,-0.059799,-0.114209,...,0.108428,0.352589,-0.713525,0.186439,-0.240328,-0.058913,0.021484,-0.288860,-1.152552,0
3,0.264984,-0.176535,0.334038,0.144265,0.248303,-0.292584,-0.058972,0.090379,-0.222964,-0.291404,...,0.183932,-0.363303,0.023249,-0.009051,-0.012115,-0.070903,0.052073,-0.343336,0.197485,0
4,-0.130587,0.556781,-0.235574,-0.475903,-0.003431,0.330739,-0.249002,-0.089374,-0.092351,0.062309,...,0.226602,0.407026,-0.770756,0.251722,-0.260993,-0.099140,-0.089544,-0.339107,-1.089323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,-0.012767,0.011839,0.007580,-0.266308,0.090122,-0.220506,-0.173810,-0.010637,-0.149298,0.053470,...,0.048565,-0.126606,-0.108115,0.028926,0.151063,-0.173255,0.213742,0.216159,0.021915,3
156,-0.085824,-0.099274,-0.088052,-0.164931,-0.080422,-0.215672,-0.060028,-0.103946,0.145708,-0.027915,...,0.200452,-0.202974,-0.293518,0.033146,0.106471,0.027705,0.160602,0.173740,-0.062390,3
157,-0.338009,0.847410,-0.252539,-0.515897,-0.064784,0.453392,-0.192018,-0.157386,0.071385,-0.096256,...,0.175021,0.577367,-0.942202,0.365627,-0.377919,-0.046364,0.116191,-0.516801,-1.421096,3
158,0.142190,0.309045,0.321782,0.026317,0.077710,-0.191903,-0.129991,0.162830,0.001927,-0.128485,...,0.351996,-0.521999,-0.120552,0.043724,0.172175,-0.260615,0.276205,0.056224,0.035223,3


In [9]:
df.to_csv("fetures.csv", index=False)

_WANDB_ Init

In [32]:
wandb.init(project="Actor_Detection_MLP")

accuracy,▁▄▅▆▆▇▇▇▇████▇██████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▄▃▃▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
epoch,299
loss,0.00042


_WANDB_ Config

In [13]:
config = wandb.config
config.learning_rate = 0.001

Dataset

In [15]:
dataset = pd.read_csv("fetures.csv")
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
0,-0.250973,0.561123,-0.053094,-0.410861,-0.018607,0.310215,-0.151722,-0.202542,-0.009447,-0.112865,...,0.094442,0.438000,-0.781172,0.274844,-0.264912,-0.059449,-0.007042,-0.383336,-1.143475,0
1,-0.245689,0.708880,-0.176685,-0.457115,-0.031878,0.404533,-0.227349,-0.208888,-0.017394,-0.034352,...,0.117269,0.471791,-0.844017,0.300227,-0.297739,-0.103941,0.042201,-0.470301,-1.371787,0
2,-0.230934,0.612900,-0.192643,-0.514945,0.050534,0.365307,-0.227889,-0.090390,-0.059799,-0.114209,...,0.108428,0.352589,-0.713525,0.186439,-0.240328,-0.058913,0.021484,-0.288860,-1.152552,0
3,0.264984,-0.176535,0.334038,0.144265,0.248303,-0.292584,-0.058972,0.090379,-0.222964,-0.291404,...,0.183932,-0.363303,0.023249,-0.009051,-0.012115,-0.070903,0.052073,-0.343336,0.197485,0
4,-0.130587,0.556781,-0.235574,-0.475903,-0.003431,0.330739,-0.249002,-0.089374,-0.092351,0.062309,...,0.226602,0.407026,-0.770756,0.251722,-0.260993,-0.099140,-0.089544,-0.339107,-1.089323,0


In [17]:
Y = dataset[['target']]
X = dataset.drop(['target'], axis=1)

X.head(3)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.250973,0.561123,-0.053094,-0.410861,-0.018607,0.310215,-0.151722,-0.202542,-0.009447,-0.112865,...,-0.956411,0.094442,0.438000,-0.781172,0.274844,-0.264912,-0.059449,-0.007042,-0.383336,-1.143475
1,-0.245689,0.708880,-0.176685,-0.457115,-0.031878,0.404533,-0.227349,-0.208888,-0.017394,-0.034352,...,-1.179397,0.117269,0.471791,-0.844017,0.300227,-0.297739,-0.103941,0.042201,-0.470301,-1.371787
2,-0.230934,0.612900,-0.192643,-0.514945,0.050534,0.365307,-0.227889,-0.090390,-0.059799,-0.114209,...,-1.007394,0.108428,0.352589,-0.713525,0.186439,-0.240328,-0.058913,0.021484,-0.288860,-1.152552


Train and Test split

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=33)

Model

In [27]:
model = tf.keras.models.Sequential([
    Dense(256, input_dim=512, activation="sigmoid"),
    Dense(512, activation="relu"),
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dense(4, activation="softmax") # Output Layer
])

In [28]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

Model Summary

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 32)                4128      
                                                                 
 dense_9 (Dense)             (None, 4)                 132       
                                                                 
Total params: 332,836
Trainable params: 332,836
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# tf.keras.utils.plot_model(model, show_shapes=True)

Train

In [30]:
model.fit(X_train, Y_train, epochs=300, callbacks=[WandbCallback()])

Epoch 1/300
4/4 [==============================] - 1s 12ms/step - loss: 1.3879 - accuracy: 0.3359 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 2/300
4/4 [==============================] - 0s 12ms/step - loss: 1.3054 - accuracy: 0.3672 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 3/300
4/4 [==============================] - 0s 11ms/step - loss: 1.2390 - accuracy: 0.4062 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 4/300
4/4 [==============================] - 0s 12ms/step - loss: 1.2077 - accuracy: 0.4062 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 5/300
4/4 [==============================] - 0s 11ms/step - loss: 1.1298 - accuracy: 0.3828 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 6/300
4/4 [==============================] - 0s 12ms/step - loss: 1.0747 - accuracy: 0.4141 - _timestamp: 1646248155.0000 - _runtime: 153.0000
Epoch 7/300
4/4 [==============================] - 0s 15ms/step - loss: 1.0062 - accuracy: 0.6016 - _times

Evaluate

In [31]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 191ms/step - loss: 0.5445 - accuracy: 0.8125


[0.5444921851158142, 0.8125]

Predict

In [43]:
embedding = DeepFace.represent(img_path="./Dataset_Actor/Pezhman_Jamshidi/5_1.jpg", model_name="ArcFace")
np.argmax(model.predict([embedding]))

1

In [44]:
embedding = DeepFace.represent(img_path="./Dataset_Actor/Golshifteh_Farahani/Golshifteh-Farahani_0.jpg", model_name="ArcFace")
np.argmax(model.predict([embedding]))

0

In [45]:
embedding = DeepFace.represent(img_path="./Dataset_Actor/Shaghayegh_Dehghan/a_0.jpg", model_name="ArcFace")
np.argmax(model.predict([embedding]))

2

In [46]:
embedding = DeepFace.represent(img_path="./Dataset_Actor/Shahram_Nazeri/24_0.jpg", model_name="ArcFace")
np.argmax(model.predict([embedding]))

3